In [1]:
import pennylane as qml
import torch
import torch.nn as nn
import numpy as np
import time
import os

In [2]:
torch.manual_seed(42)
np.random.seed(42)

In [3]:
# 加载数据
data = np.load('../DataSpace/csi_cmri/CSI_channel_30km.npy')  # shape=(80000, 2560)

In [159]:
# 数据划分参数
TOTAL_SAMPLES = 80000
TRAIN_RATIO = 0.70    # 70% 训练
VAL_RATIO = 0.15      # 15% 验证  
TEST_RATIO = 0.15     # 15% 测试

# 设置量子线路参数
INPUT_DIM = data.shape[1]
OUTPUT_DIM = 256
ORG_QUBITS = int(np.ceil(np.log2(INPUT_DIM)))
TAR_QUBITS = int(np.ceil(np.log2(OUTPUT_DIM)))
ALL_QUBITS = ORG_QUBITS + 1
ANS_QUBITS = ORG_QUBITS - TAR_QUBITS

In [160]:
C_WEIGHT = torch.randn(INPUT_DIM, OUTPUT_DIM, requires_grad=True)
C_BIAS = torch.randn(1, OUTPUT_DIM, requires_grad=True)

LAYERS = 4
#DIST = [1] * LAYERS
DIST = [1, 2, 3, 4]
Q_WEIGHT = torch.randn(LAYERS, ALL_QUBITS, 3, requires_grad=True)

In [161]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def normlize(x):
    norm = torch.norm(x)
    if norm == 0:
        return x
    return x / norm

def dense_layer(x, c_weight, c_bias):
    if isinstance(x, np.ndarray):
        x = torch.from_numpy(x).float()
    output = torch.matmul(x, c_weight) + c_bias
    output = sigmoid(output)
    output = normlize(output[0])  # 确保输出是一维的
    return output

In [162]:
# FRQI编码量子线路
def frqi_encoder(qubits, params, target_wire=0):
    # 对数据量子比特应用Hadamard门创建叠加态
    for i in range(1, qubits+1):
        qml.Hadamard(wires=i)
    # 使用受控PauliY旋转进行编码
    for index in range(min(2**qubits, len(params))):
        binary_str = bin(index)[2:].zfill(qubits)
        bits = [int(bit) for bit in binary_str]
        bits.reverse()
        qml.ctrl(qml.RY, control=range(1, qubits + 1), control_values=bits)(2*params[index], wires=target_wire)

In [163]:
# 构造强纠缠层ansatz线路
def strong_entangling_ansatz(q_weight, dist=DIST):
    if q_weight.shape != (LAYERS, ALL_QUBITS, 3):
        print("Wrong q_weight!")
        return
    if len(dist) != LAYERS:
        print("Wrong distance!")
        return
    for i_layer in range(LAYERS):
        for i_qubit in range(ALL_QUBITS):
            qml.Rot(q_weight[i_layer, i_qubit, 0], q_weight[i_layer, i_qubit, 1], q_weight[i_layer, i_qubit, 2], wires=i_qubit)
        for i_qubit in range(ALL_QUBITS):
            target_qubit = (i_qubit + dist[i_layer]) % ALL_QUBITS
            qml.CNOT(wires=[i_qubit, target_qubit])

In [173]:
# 完整的经典-量子混合神经网络
dev = qml.device("lightning.qubit", wires=ALL_QUBITS)
@qml.qnode(dev, interface = "torch")
def cir(sample, c_weight, c_bias, q_weight=0):
    y = dense_layer(sample, c_weight, c_bias)
    frqi_encoder(qubits=TAR_QUBITS, params=y)
    strong_entangling_ansatz(q_weight)
    x = -sample
    frqi_encoder(qubits=ORG_QUBITS, params=x)    
    Ham = qml.Hamiltonian([-1], [qml.PauliZ(0)])
    return qml.expval(Ham)

In [174]:
loss = cir(data[6], C_WEIGHT, C_BIAS, Q_WEIGHT)
print(f"The loss is: {loss}")
loss.backward()

The loss is: -0.00895815622061491


In [175]:
C_WEIGHT.grad

tensor([[-7.1343e-11, -1.5368e-10,  7.6508e-10,  ..., -5.1630e-10,
         -2.4357e-10, -6.6035e-10],
        [ 6.7114e-13,  1.4457e-12, -7.1973e-12,  ...,  4.8569e-12,
          2.2913e-12,  6.2121e-12],
        [ 5.9163e-12,  1.2744e-11, -6.3446e-11,  ...,  4.2815e-11,
          2.0199e-11,  5.4761e-11],
        ...,
        [-1.1723e-11, -2.5253e-11,  1.2572e-10,  ..., -8.4839e-11,
         -4.0025e-11, -1.0851e-10],
        [-5.7892e-11, -1.2470e-10,  6.2083e-10,  ..., -4.1895e-10,
         -1.9765e-10, -5.3585e-10],
        [ 1.7884e-11,  3.8524e-11, -1.9179e-10,  ...,  1.2942e-10,
          6.1058e-11,  1.6553e-10]])

In [167]:
Q_WEIGHT.grad

tensor([[[ 2.0063e-04, -6.5499e-03, -5.3888e-03],
         [-8.3381e-03, -3.5901e-03, -2.0206e-03],
         [ 1.7688e-03, -4.0166e-03,  1.7489e-03],
         [-7.5990e-04,  1.0726e-02, -8.6820e-04],
         [ 1.4542e-02,  7.2888e-03,  1.6128e-02],
         [ 3.0412e-03,  1.3656e-03,  4.1903e-03],
         [ 6.8463e-03, -9.9732e-04,  2.6837e-03],
         [-1.3668e-02, -1.3480e-03, -7.0739e-03],
         [-2.6332e-03, -1.0183e-02, -6.5374e-03],
         [ 2.7318e-17, -9.4714e-03, -2.6770e-03],
         [ 1.1196e-17, -2.1501e-04, -4.6914e-03],
         [ 3.3871e-18,  1.0789e-02,  8.7505e-04],
         [ 2.4004e-17,  2.7305e-02, -8.2411e-04]],

        [[-6.2404e-04, -5.2920e-03,  1.6060e-04],
         [-6.8310e-03,  6.3233e-03,  3.1347e-03],
         [ 7.1265e-03, -4.7278e-04,  1.6064e-02],
         [-5.3223e-03,  7.3407e-03, -3.2146e-03],
         [ 3.3255e-03, -7.9815e-03, -1.3669e-03],
         [-5.9056e-03, -8.4656e-03, -4.5166e-03],
         [ 5.7124e-03,  1.1189e-02,  3.2227e-03]

In [168]:
C_BIAS.grad

tensor([[-2.3279e-06, -5.0146e-06,  2.4965e-05,  1.3796e-05, -3.9634e-05,
          1.1005e-05,  1.4766e-05,  1.9781e-05, -1.0826e-05, -7.8464e-06,
          1.2619e-05,  4.4709e-05,  1.0246e-05, -2.1675e-05, -5.7728e-06,
          7.2340e-05, -2.1491e-05, -2.4441e-05,  1.5076e-05, -5.4948e-05,
          1.8378e-05, -6.4029e-06,  5.2290e-05, -1.3263e-05, -1.0429e-05,
          2.5670e-05,  1.5091e-05,  4.0452e-05, -2.1902e-05,  8.2316e-06,
         -4.8324e-05, -1.5278e-05, -2.1401e-05, -5.6739e-06,  1.1850e-05,
          5.3217e-06, -9.9015e-06,  9.3964e-06, -2.3278e-05, -8.5764e-06,
          1.3644e-05, -1.4262e-05, -6.0007e-05,  6.4516e-06,  4.1631e-05,
          3.1256e-06,  2.1551e-06,  2.9055e-06,  1.1469e-05,  3.2069e-05,
          2.2579e-05,  1.0452e-05,  1.0233e-05,  1.4786e-05,  1.2289e-05,
          4.2790e-05, -7.8419e-06, -1.2297e-05,  3.5241e-05, -6.8256e-06,
          1.3522e-05, -5.8135e-06, -7.0289e-07,  1.0333e-05,  1.4887e-05,
         -1.9261e-05, -3.2979e-05, -4.

In [178]:
data[5]

array([-1.28871e-05,  1.03035e-05,  1.78284e-07, ..., -6.89643e-06,
       -1.55457e-05,  9.68996e-06])

In [180]:
base_path = "~/DataSpace/"
file_name = "file1.png"
base_path + file_name

'~/DataSpace/file1.png'

In [3]:
[-1] * 5

[-1, -1, -1, -1, -1]

In [9]:
qml.Hamiltonian([-1.0]*5, [qml.PauliZ(i) for i in range(5)])

(
    -1.0 * Z(0)
  + -1.0 * Z(1)
  + -1.0 * Z(2)
  + -1.0 * Z(3)
  + -1.0 * Z(4)
)

In [11]:
obs = qml.PauliZ(0)
for i in range(1, 3):
    obs = obs @ qml.PauliZ(i)
qml.Hamiltonian([-1.0], [obs])

-1.0 * (Z(0) @ Z(1) @ Z(2))